In [2]:
import cv2
import pywt
import numpy as np
from tqdm import tqdm
import os
"""
双边滤波：
sigma_color 和 sigma_space 设置较低值，避免过度模糊。
Sobel 边缘增强：
使用 Sobel 算子来计算图像的梯度并增强边缘细节。通过 cv2.addWeighted 方法融合边缘图像和原始图像，避免图像细节丢失。
小波变换：
调整小波变换的阈值（threshold_value=15），减少去噪强度，以保留更多细节，特别是边缘信息。
拉普拉斯锐化：
拉普拉斯锐化的强度（alpha=0.5）适度，避免过度锐化。
"""
def bilateral_filtering(image):
    """
    对输入图像进行双边滤波
    """
    # 减小双边滤波的平滑程度，避免过度模糊
    diameter = 9
    sigma_color = 30  # 更低的色差标准差，减少平滑强度
    sigma_space = 30  # 更低的坐标空间标准差
    return cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

def wavelet_transform(image, thresholding_method='soft', threshold_value=15):
    """
    对输入图像进行小波变换及更完善的处理后重构
    """
    # 使用不同的小波基，如db1或haar来保留更多细节
    wavelet = 'db1'
    mode = 'symmetric'

    # 进行二维小波分解
    coeffs = pywt.dwt2(image, wavelet, mode=mode)
    cA, (cH, cV, cD) = coeffs

    # 对小波系数进行阈值处理（以去噪并保留重要细节）
    cA_thresholded = threshold_wavelet_coefficients(cA, thresholding_method, threshold_value)
    cH_thresholded = threshold_wavelet_coefficients(cH, thresholding_method, threshold_value)
    cV_thresholded = threshold_wavelet_coefficients(cV, thresholding_method, threshold_value)
    cD_thresholded = threshold_wavelet_coefficients(cD, thresholding_method, threshold_value)

    new_coeffs = cA_thresholded, (cH_thresholded, cV_thresholded, cD_thresholded)

    return pywt.idwt2(new_coeffs, wavelet, mode=mode)

def threshold_wavelet_coefficients(coeffs, method='soft', threshold=15):
    """
    对小波系数按照指定方法和阈值进行处理
    """
    if method == 'soft':
        # 软阈值处理
        sgn = np.sign(coeffs)
        abs_coeffs = np.abs(coeffs)
        thresholded_abs = np.where(abs_coeffs > threshold, abs_coeffs - threshold, 0)
        return sgn * thresholded_abs
    elif method == 'hard':
        # 硬阈值处理
        return coeffs * (np.abs(coeffs) > threshold)
    else:
        raise ValueError("Invalid thresholding method. Choose either 'soft' or 'hard'.")

def sobel_edge_enhancement(image):
    """
    使用Sobel算子进行边缘增强
    """
    # Sobel算子计算梯度
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobel_edges = cv2.magnitude(sobel_x, sobel_y)
    
    # 将sobel_edges的范围限制在[0, 255]，并将数据类型转换为 uint8
    sobel_edges = np.uint8(np.clip(sobel_edges, 0, 255))  # 限制边缘强度

    # 将图像类型转换为 uint8，以便与 sobel_edges 进行加权操作
    image = np.uint8(np.clip(image, 0, 255))  # 确保 image 是 uint8 类型

    # 融合边缘图像与原始图像
    enhanced_image = cv2.addWeighted(image, 0.7, sobel_edges, 0.3, 0)  # 调整权重，增强边缘
    return enhanced_image


def laplacian_sharpening(image, alpha=0.5):
    """
    对输入图像进行拉普拉斯锐化，避免过度锐化
    """
    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])
    sharpened = cv2.filter2D(image, -1, kernel)
    # 锐化增强，但避免过度锐化
    sharpened_image = cv2.addWeighted(image, 1, sharpened, alpha, 0)
    return sharpened_image

def process_images(input_folder, output_folder):
    """
    批量处理图像的主函数
    """
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
    for image_file in tqdm(image_files):
        # 读取图像
        image_path = os.path.join(input_folder, image_file)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue
        # 双边滤波
        filtered_image = bilateral_filtering(image)
        # 小波变换
        wavelet_image = wavelet_transform(filtered_image)
        # 边缘增强
        edge_enhanced_image = sobel_edge_enhancement(wavelet_image)
        # 拉普拉斯锐化
        sharpened_image = laplacian_sharpening(edge_enhanced_image)

        # 保存处理后的图像到新文件夹
        output_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_path, sharpened_image)

if __name__ == "__main__":
    input_folder = r"E:\pycharm_pro\Unet\MyUnet\train_images"  # 替换为实际的输入图像文件夹路径
    output_folder = r"E:\pycharm_pro\Unet\MyUnet\preprocessing\1228pre3"  # 替换为实际的输出图像文件夹路径
    process_images(input_folder, output_folder)


100%|██████████| 2000/2000 [00:57<00:00, 34.87it/s]


RGB process

In [3]:
import cv2
import pywt
import numpy as np
from tqdm import tqdm
import os

def bilateral_filtering(image):
    """
    对输入图像进行双边滤波（RGB图像）
    """
    # 调整双边滤波的平滑强度
    diameter = 9
    sigma_color = 30  # 更低的色差标准差
    sigma_space = 30  # 更低的坐标空间标准差
    return cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

def wavelet_transform(image, thresholding_method='soft', threshold_value=15):
    """
    对输入图像进行小波变换及更完善的处理后重构
    """
    wavelet = 'db1'
    mode = 'symmetric'

    # 处理每个颜色通道
    channels = cv2.split(image)
    processed_channels = []

    for channel in channels:
        coeffs = pywt.dwt2(channel, wavelet, mode=mode)
        cA, (cH, cV, cD) = coeffs

        cA_thresholded = threshold_wavelet_coefficients(cA, thresholding_method, threshold_value)
        cH_thresholded = threshold_wavelet_coefficients(cH, thresholding_method, threshold_value)
        cV_thresholded = threshold_wavelet_coefficients(cV, thresholding_method, threshold_value)
        cD_thresholded = threshold_wavelet_coefficients(cD, thresholding_method, threshold_value)

        new_coeffs = cA_thresholded, (cH_thresholded, cV_thresholded, cD_thresholded)
        processed_channel = pywt.idwt2(new_coeffs, wavelet, mode=mode)
        processed_channels.append(processed_channel)

    # 合并处理后的通道
    return cv2.merge(processed_channels)

def threshold_wavelet_coefficients(coeffs, method='soft', threshold=15):
    """
    对小波系数按照指定方法和阈值进行处理
    """
    if method == 'soft':
        sgn = np.sign(coeffs)
        abs_coeffs = np.abs(coeffs)
        thresholded_abs = np.where(abs_coeffs > threshold, abs_coeffs - threshold, 0)
        return sgn * thresholded_abs
    elif method == 'hard':
        return coeffs * (np.abs(coeffs) > threshold)
    else:
        raise ValueError("Invalid thresholding method. Choose either 'soft' or 'hard'.")

def sobel_edge_enhancement(image):
    """
    使用Sobel算子进行边缘增强（RGB图像）
    """
    # 处理每个颜色通道
    channels = cv2.split(image)
    enhanced_channels = []

    for channel in channels:
        sobel_x = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=3)
        sobel_edges = cv2.magnitude(sobel_x, sobel_y)

        # 限制sobel_edges的值，并转换为uint8
        sobel_edges = np.uint8(np.clip(sobel_edges, 0, 255))

        # 将图像类型转换为 uint8，以便与 sobel_edges 进行加权操作
        channel = np.uint8(np.clip(channel, 0, 255))  # 确保channel是uint8类型

        # 融合边缘图像与原始通道
        enhanced_channel = cv2.addWeighted(channel, 0.7, sobel_edges, 0.3, 0)
        enhanced_channels.append(enhanced_channel)

    # 合并增强后的通道
    return cv2.merge(enhanced_channels)

def laplacian_sharpening(image, alpha=0.5):
    """
    对输入图像进行拉普拉斯锐化（RGB图像）
    """
    # 处理每个颜色通道
    channels = cv2.split(image)
    sharpened_channels = []

    for channel in channels:
        kernel = np.array([[0, 1, 0],
                           [1, -4, 1],
                           [0, 1, 0]])
        sharpened = cv2.filter2D(channel, -1, kernel)
        sharpened_channel = cv2.addWeighted(channel, 1, sharpened, alpha, 0)
        sharpened_channels.append(sharpened_channel)

    # 合并锐化后的通道
    return cv2.merge(sharpened_channels)

def process_images(input_folder, output_folder):
    """
    批量处理图像的主函数
    """
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]
    for image_file in tqdm(image_files):
        # 读取RGB图像
        image_path = os.path.join(input_folder, image_file)
        image = cv2.imread(image_path)  # 默认以BGR格式读取
        if image is None:
            continue

        # 转换为RGB格式
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 双边滤波
        filtered_image = bilateral_filtering(image_rgb)

        # 小波变换
        wavelet_image = wavelet_transform(filtered_image)

        # 边缘增强
        edge_enhanced_image = sobel_edge_enhancement(wavelet_image)

        # 拉普拉斯锐化
        sharpened_image = laplacian_sharpening(edge_enhanced_image)

        # 转换回BGR格式保存
        output_image_bgr = cv2.cvtColor(sharpened_image, cv2.COLOR_RGB2BGR)
        output_path = os.path.join(output_folder, image_file)
        cv2.imwrite(output_path, output_image_bgr)

if __name__ == "__main__":
    input_folder = r"E:\pycharm_pro\Unet\MyUnet\train_images"  # 替换为实际的输入图像文件夹路径
    output_folder = r"E:\pycharm_pro\Unet\MyUnet\preprocessing\1228pre4"  # 替换为实际的输出图像文件夹路径
    process_images(input_folder, output_folder)


100%|██████████| 2000/2000 [03:11<00:00, 10.45it/s]
